In [2]:
import numpy as np
import xarray as xr
import pandas as pd
import numcodecs
from dask.distributed import Client, progress, LocalCluster
import zarr
import glob

In [2]:
cluster = LocalCluster()
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:39221 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 32 Memory: 132.24 GB


In [11]:
#xr.open_zarr('/data/mursst_netcdf/zarr/1x4000x8000')
xr.open_mfdataset(['/data/mursst_netcdf/v4.1/2015/152/20150601090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc'])

<ipython-input-11-aaea35e59654>:2: FutureWarning: In xarray version 0.15 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  xr.open_mfdataset(['/data/mursst_netcdf/v4.1/2015/152/20150601090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc'])
/opt/conda/envs/netcdf_to_zarr/lib/python3.8/site-packages/xarray/backends/api.py:926: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  combined = auto_combine(


<xarray.Dataset>
Dimensions:           (lat: 17999, lon: 36000, time: 1)
Coordinates:
  * time              (time) datetime64[ns] 2015-06-01T09:00:00
  * lat               (lat) float32 -89.99 -89.98 -89.97 ... 89.97 89.98 89.99
  * lon               (lon) float32 -179.99 -179.98 -179.97 ... 179.99 180.0
Data variables:
    analysed_sst      (time, lat, lon) float32 dask.array<chunksize=(1, 17999, 36000), meta=np.ndarray>
    analysis_error    (time, lat, lon) float32 dask.array<chunksize=(1, 17999, 36000), meta=np.ndarray>
    mask              (time, lat, lon) float32 dask.array<chunksize=(1, 17999, 36000), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float32 dask.array<chunksize=(1, 17999, 36000), meta=np.ndarray>
Attributes:
    Conventions:                CF-1.5
    title:                      Daily MUR SST, Final product
    summary:                    A merged, multi-sensor L4 Foundation SST anal...
    references:                 http://podaac.jpl.nasa.gov/Multi-scale_Ultra-...
    institution:                Jet Propulsion Laboratory
    history:                    created at nominal 4-day latency; replaced nr...
    comment:                    MUR = "Multi-scale Ultra-high Reolution"
    license:                    These data are available free of charge under...
    id:                         MUR-JPL-L4-GLOB-v04.1
    naming_authority:           org.ghrsst
    product_version:            04.1
    uuid:                       27665bc0-d5fc-11e1-9b23-0800200c9a66
    gds_version_id:             2.0
    netcdf_version_id:          4.1
    date_created:               20150605T013045Z
    start_time:                 20150601T090000Z
    stop_time:                  20150601T090000Z
    time_coverage_start:        20150531T210000Z
    time_coverage_end:          20150601T210000Z
    file_quality_level:         1
    source:                     AVHRR19_G-NAVO, AVHRR_METOP_A-EUMETSAT, MODIS...
    platform:                   Aqua, DMSP, NOAA-POES, Suomi-NPP, Terra
    sensor:                     AMSR-E, AVHRR, MODIS, SSM/I, VIIRS, in-situ
    Metadata_Conventions:       Unidata Observation Dataset v1.0
    metadata_link:              http://podaac.jpl.nasa.gov/ws/metadata/datase...
    keywords:                   Oceans > Ocean Temperature > Sea Surface Temp...
    keywords_vocabulary:        NASA Global Change Master Directory (GCMD) Sc...
    standard_name_vocabulary:   NetCDF Climate and Forecast (CF) Metadata Con...
    southernmost_latitude:      -90.0
    northernmost_latitude:      90.0
    westernmost_longitude:      -180.0
    easternmost_longitude:      180.0
    spatial_resolution:         0.01 degrees
    geospatial_lat_units:       degrees north
    geospatial_lat_resolution:  0.01 degrees
    geospatial_lon_units:       degrees east
    geospatial_lon_resolution:  0.01 degrees
    acknowledgment:             Please acknowledge the use of these data with...
    creator_name:               JPL MUR SST project
    creator_email:              ghrsst@podaac.jpl.nasa.gov
    creator_url:                http://mur.jpl.nasa.gov
    project:                    NASA Making Earth Science Data Records for Us...
    publisher_name:             GHRSST Project Office
    publisher_url:              http://www.ghrsst.org
    publisher_email:            ghrsst-po@nceo.ac.uk
    processing_level:           L4
    cdm_data_type:              grid

In [3]:
def generate_file_list(start_doy, end_doy):   
    """
    Given a start day and end end day, generate a list of file locations.
    Assumes a 'prefix' and 'year' variables have already been defined.
    'Prefix' should be a local directory or http url and path.
    'Year' should be a 4 digit year.
    """
    days_of_year = list(range(start_doy, end_doy))
    fileObjs = []

    for doy in days_of_year:
        file = glob.glob(f"{prefix}/{year}/{doy}/*.nc")[0]
        fileObjs.append(file)
    return fileObjs

In [4]:
year = 2006
start_doy = 152
end_doy = 252
prefix = '/data/mursst_netcdf/v4.1/'
zarr_prefix = '/data/mursst_netcdf/zarr/'
fileObjs = generate_file_list(start_doy, end_doy)

In [ ]:
import time
from datetime import datetime
chunk_opts = [
    {'time': 1, 'lat': 4000, 'lon': 8000, 'tests': { 'netcdf': {}, 'zarr': {} }},
    {'time': 5, 'lat': 1800, 'lon': 3600, 'tests': { 'netcdf': {}, 'zarr': {} }},
    {'time': 10, 'lat': 1400, 'lon': 2800, 'tests': { 'netcdf': {}, 'zarr': {} }}    
]

time_slice = slice(datetime.strptime(f"{year}-06-01", '%Y-%m-%d'), datetime.strptime(f"{year}-06-05", '%Y-%m-%d'))
lat_slice = slice(10, 50)
lon_slice = slice(-150, -100)

for chunks in chunk_opts:
    zarr_subdirectory = 'x'.join(map(str, list(chunks.values())[0:3]))
    print(zarr_subdirectory)
    ###############################
    ###############################
    # NETCDF Tests
    ###############################
    # Test opening the store
    ###############################
    s = time.time()
    ds_netcdf = xr.open_mfdataset(fileObjs, chunks=chunks, parallel=True, combine='by_coords')
    test_duration = time.time() - s
    print(f"time to open files for chunking {zarr_subdirectory}: {test_duration} seconds")
    chunks['tests']['netcdf']['open'] = test_duration
    ###############################
    # Test averaging globally for a 5 day time slice
    ###############################
    s = time.time()
    ds_netcdf.analysed_sst.sel(time=time_slice).mean({'lat','lon'}).values
    test_duration = time.time() - s
    print(f"time to calculate mean for 5 day time slice for chunking {zarr_subdirectory}: {test_duration} seconds")
    chunks['tests']['netcdf']['global 5 day slice'] = test_duration
    ###############################
    # Test averaging for 100 days for a lat/lon bbox
    ###############################
    s = time.time()
    ds_netcdf.analysed_sst.sel(lat=lat_slice, lon=lon_slice).mean({'lat','lon'}).values
    test_duration = time.time() - s
    print(f"time to calculate mean for lat/lon slice for chunking {zarr_subdirectory}: {test_duration} seconds")
    chunks['tests']['netcdf']['bbox 100 day'] = test_duration
    ###############################
    ###############################
    ###############################
    # Zarr Tests
    ###############################
    # Test opening the store
    ###############################
    s = time.time()
    ds_zarr = xr.open_zarr(f"{zarr_store}{zarr_prefix}")
    test_duration = time.time() - s
    print(f"time to open files for chunking {zarr_subdirectory}: {test_duration} seconds")
    chunks['tests']['zarr']['open'] = test_duration
    ###############################
    # Test averaging globally for a 5 day time slice
    ###############################
    s = time.time()
    ds_zarr.analysed_sst.sel(time=time_slice).mean({'lat','lon'}).values
    test_duration = time.time() - s
    print(f"time to calculate mean for 5 day time slice for chunking {chunks}: {test_duration} seconds")
    chunks['tests']['zarr']['global 5 day slice'] = test_duration
    ###############################
    # Test averaging for 100 days for a lat/lon bbox
    ###############################
    s = time.time()
    ds_zarr.analysed_sst.sel(lat=lat_slice, lon=lon_slice).mean({'lat','lon'}).values
    test_duration = time.time() - s
    print(f"time to calculate mean for lat/lon slice for chunking {chunks}: {test_duration} seconds")      
    chunks['tests']['zarr']['bbox 100 day'] = test_duration